# **Text summarization**:
## **There are two major types of text summarization**:
  -	Extractive: summaries consist of text from the original document.   Extractive summaries are relatively easy to achieve. The whole concept is to find the relevant parts of the input text. 
  
  -	Abstractive: summaries can contain novel sequences of text not necessarily taken from the original text. Abstractive summaries are harder to achieve, even generating a very simple text is quite a challenge.


In this project we will be looking at extractive summarization in details and we will touch on abstractive summarization at the end for a demo purpose.


# **Section 1**
detailed implemention of every step of the algorithm concept

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1- Text summarization using vectors - TF-IDF**
### **Steps:**
        - Split the text into sentences
        - Score each sentence
        - Rank each sentence by those scores
        - Summary = top scoring sentences
     

## **let start by importing the needed libraries**

In [2]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import data_table
data_table.enable_dataframe_formatter()

## **Downloading al the necessary files from NLTK**

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## **Loading-in the input dataset**

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Working_Dir/NLP-Project/Code-data/argusnews.csv')
#df = pd.read_csv('bbc_text_cls.csv')

## **Printing out the data to see the structure of the dataset and understand what kind of data we are dealing with**

In [5]:
df.head()

title   publish_date  \
0  CSU predicts above-normal Atlantic hurricane a...  07 April 2022   
1  New plants expand Brazil’s power capacity by 3...  07 April 2022   
2      New Petrobras CEO, chairman confirmed: Update  14 April 2022   
3   US regulator plans hearing on rail services woes  07 April 2022   
4               EPA reverses Trump-era SRE decisions  07 April 2022   

                           writer  \
0               By Nola Z Valente   
1             By Rebecca Gompertz   
2              By Nathan Walters    
3                  By Abby Caplan   
4  By Dylan Chase and Thom Dwyer    

                                             artical  \
0  The 2022 Atlantic hurricane season could be mo...   
1  Brazil increased its power generation capacity...   
2  Updates with comments from Coelho. Shareholder...   
3  US rail regulators will hold a hearing on 26-2...   
4  The US Environmental Protection Agency today d...   

                                        artical_html  \
0  <p class="lead">The 2022 Atlantic hurricane se...   
1  <p class="lead">Brazil increased its power gen...   
2   <p><i>Updates with comments from Coelho.</i></p>   
3  <p class="lead">US rail regulators will hold a...   
4  <p class="lead">The US Environmental Protectio...   

                                                 url  
0  https://www.argusmedia.com/en/news/2320047-csu...  
1  https://www.argusmedia.com/en/news/2319883-new...  
2  https://www.argusmedia.com/en/news/2322507-new...  
3  https://www.argusmedia.com/en/news/2320157-us-...  
4  https://www.argusmedia.com/en/news/2320164-epa...

## **Creating a document by choosing one of the text within the dataset randomly or otherwise**

In [6]:
#doc = df[df.labels == 'business']['text'].sample(random_state=42)
#doc = df[df.publish_date == '07 April 2022']['artical'].sample(random_state=42)
doc = df[df.title == 'CSU predicts above-normal Atlantic hurricane activity']['artical'].sample(random_state=42)

## **Define a wrap function to help us print the result more nicely**

In [7]:
def wrap(x):
    return textwrap.fill(x, replace_whitespace = False, fix_sentence_endings= True)

## **Print the chosen text to see what we have**

In [8]:
print(wrap(doc.iloc[0]))

The 2022 Atlantic hurricane season could be more active than usual as
sea temperatures across the tropical Atlantic are higher than normal,
Colorado State University (CSU) said in its annual outlook today.
Another factor for the above-average prediction is the absence of El
Niño, according to CSU researchers.  The six-month long hurricane
season should have 19 named storms, nine of which could become
hurricanes and four that could turn into major hurricanes of Category
3 or higher, according to the outlook.  This would be the seventh
consecutive above-average Atlantic hurricane season.  The average for
tropical cyclones in the Atlantic between 1991 and 2020 is seven
hurricanes, three of them major, and 14 tropical storms, according to
the US National Oceanic and Atmospheric Administration.  The 2022
hurricane activity should be about 130pc above the average season from
1991 to 2020, compared with 2021's hurricane season that was up by
120pc.  Hurricanes have had less impact on energy m

## **Tokenize our document or text into senetnces**

In [9]:
#sents = nltk.sent_tokenize(doc.iloc[0].split("\n", 1)[1])
sents = nltk.sent_tokenize(doc.iloc[0])

## **create TF-IDF vectorizer object note that we are using English stopwords and l1 normlization this insure that we will not be bias toward longer sentences which in this case would have a higher score. simply due to the fact that they contain more words**

In [10]:
featurizer = TfidfVectorizer(
    stop_words= stopwords.words('english'), norm= 'l1')

## **next we call fit transform which give us back our TF-IDF matrix (X)** 

In [11]:
X = featurizer.fit_transform(sents)

## **let create a function to score a sentence given it's TF-IDF representation. As can be seen it take a single row from the TF-IDF matrix. Only non zero values are considered once you have those values a mean function is applied** 

In [12]:
def get_sentence_score(tfidf_row):
    # return the average of the non-zero values
    # of the tf-idf vector representation of a sentence
    x = tfidf_row[tfidf_row != 0]
    return x.mean()

## **Start the summarization process using functional programming**
### **build a summarization function that include some extra steps such as scoring and sorting. looping through the firts 5 elements**

In [13]:
def summarize(text):
    # extract sentences
    sents = nltk.sent_tokenize(text)
    
    #perform tf-idf
    X = featurizer.fit_transform(sents)
    
    #compute scores for each sentence
    scores = np.zeros(len(sents))
    for i in range(len(sents)):
        score = get_sentence_score(X[i,:])
        scores[i] = score
    
    #sort the scores
    sort_idx = np.argsort(-scores)
    
    #print summary
    for i in sort_idx[:5]:
        print(wrap("%.2f: %s" % (scores[i], sents[i])))

## **pick the chosen text to be summarized based on its title (unique title for each text)**

In [14]:
#doc = df[df.labels == 'business']['text'].sample(random_state=123)
doc = df[df.title == 'CSU predicts above-normal Atlantic hurricane activity']['artical'].sample(random_state=123)

## **Apply the summarization function of the chosen text and the summary**

In [15]:
#summarize(doc.iloc[0].split("\n", 1)[1])
summarize(doc.iloc[0])

0.14: This would be the seventh consecutive above-average Atlantic
hurricane season.
0.12: The CSU Tropical Meterology Project is based on about 40 years
of data.
0.10: Another factor for the above-average prediction is the absence
of El Niño, according to CSU researchers.
0.09: The 2022 hurricane activity should be about 130pc above the
average season from 1991 to 2020, compared with 2021's hurricane
season that was up by 120pc.
0.08: The Atlantic hurricane season begins 1 June and ends on 30
November, but storms can form outside of that time.


## **2 - Luhn algorithm**
- https://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf
- http://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf
- https://www.cs.cmu.edu/~nasmith/LS2/das-martins.07.pdf

## **Preparing the environment**

In [16]:
import re
import nltk
import string
import heapq

In [17]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
original_text = """The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.
                   Another factor for the above-average prediction is the absence of ElNiño, according to CSU researchers.  
                   The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.  
                   This would be the seventh consecutive above-average Atlantic hurricane season.  
                   The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration.  
                   The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc.  
                   Hurricanes have had less impact on energy markets over the years because a greater share of US output comes from onshore.  
                   Storms can disrupt operations at power plants, limiting demand for crude and natural gas, and potentially leading to higher prices for gasoline and other refined products.  
                   The Atlantic hurricane season begins 1 June and ends on 30 November, but storms can form outside of that time. 
                   The CSU Tropical Meterology Project is based on about 40 years of data."""
original_text = re.sub(r'\s+', ' ', original_text)
original_text

"The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today. Another factor for the above-average prediction is the absence of ElNiño, according to CSU researchers. The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook. This would be the seventh consecutive above-average Atlantic hurricane season. The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration. The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc. Hurricanes have had less impact on energy market

In [19]:
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [20]:
stopwords.append('explanation')

In [21]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [22]:
def preprocess(text):
  formatted_text = text.lower()
  tokens = []
  for token in nltk.word_tokenize(formatted_text):
    tokens.append(token)
  tokens = [word for word in tokens if word not in stopwords and word not in string.punctuation]
  formatted_text = ' '.join(element for element in tokens)

  return formatted_text

In [23]:
formatted_text = preprocess(original_text)
formatted_text

"2022 atlantic hurricane season could active usual sea temperatures across tropical atlantic higher normal colorado state university csu said annual outlook today another factor above-average prediction absence elniño according csu researchers six-month long hurricane season 19 named storms nine could become hurricanes four could turn major hurricanes category 3 higher according outlook would seventh consecutive above-average atlantic hurricane season average tropical cyclones atlantic 1991 2020 seven hurricanes three major 14 tropical storms according us national oceanic atmospheric administration 2022 hurricane activity 130pc average season 1991 2020 compared 2021 's hurricane season 120pc hurricanes less impact energy markets years greater share us output comes onshore storms disrupt operations power plants limiting demand crude natural gas potentially leading higher prices gasoline refined products atlantic hurricane season begins 1 june ends 30 november storms form outside time cs

## **Function to calculate sentences score**

In [24]:
def calculate_sentences_score(sentences, important_words, distance):
  scores = []
  sentence_index = 0

  for sentence in [nltk.word_tokenize(sentence) for sentence in sentences]:
    #print('------------')
    #print(sentence)

    word_index = []
    for word in important_words:
      #print(word)
      try:
        word_index.append(sentence.index(word))
      except ValueError:
        pass

    word_index.sort()
    #print(word_index)

    if len(word_index) == 0:
      continue

    # [0, 1, 5]
    groups_list = []
    group = [word_index[0]]
    i = 1 # 3
    while i < len(word_index): # 3
      # first execution: 1 - 0 = 1
      # second execution: 2 - 1 = 1
      if word_index[i] - word_index[i - 1] < distance:
        group.append(word_index[i])
        #print('group', group)
      else:
        groups_list.append(group[:])
        group = [word_index[i]]
        #print('group', group)
      i += 1
    groups_list.append(group)
    #print('all groups', groups_list)

    max_group_score = 0
    for g in groups_list:
      #print(g)
      important_words_in_group = len(g)
      total_words_in_group = g[-1] - g[0] + 1
      score = 1.0 * important_words_in_group**2 / total_words_in_group
      #print('group score', score)

      if score > max_group_score:
        max_group_score = score

    scores.append((max_group_score, sentence_index))
    sentence_index += 1

  #print('final scores', scores)
  return scores

## **Function to summarize the texts**

In [25]:
def summarize(text, top_n_words, distance, number_of_sentences, percentage = 0):
  original_sentences = [sentence for sentence in nltk.sent_tokenize(text)]
  #print(original_sentences)
  formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
  #print(formatted_sentences)
  words = [word for sentence in formatted_sentences for word in nltk.word_tokenize(sentence)]
  #print(words)
  frequency = nltk.FreqDist(words)
  #print(frequency)
  #return frequency
  top_n_words = [word[0] for word in frequency.most_common(top_n_words)]
  #print(top_n_words)
  sentences_score = calculate_sentences_score(formatted_sentences, top_n_words, distance)
  #print(sentences_score)
  if percentage > 0:
    best_sentences = heapq.nlargest(int(len(formatted_sentences) * percentage), sentences_score)
  else:  
    best_sentences = heapq.nlargest(number_of_sentences, sentences_score)
  #print(best_sentences)
  best_sentences = [original_sentences[i] for (score, i) in best_sentences]
  #print(best_sentences)
  return original_sentences, best_sentences, sentences_score

In [26]:
original_sentences, best_sentences, sentences_score = summarize(original_text, 5, 2, 3)

In [27]:
original_sentences

['The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.',
 'Another factor for the above-average prediction is the absence of ElNiño, according to CSU researchers.',
 'The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.',
 'This would be the seventh consecutive above-average Atlantic hurricane season.',
 'The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration.',
 "The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc.",
 'Hurricanes have had les

In [28]:
best_sentences

['Hurricanes have had less impact on energy markets over the years because a greater share of US output comes from onshore.',
 'The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.',
 'The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.']

In [29]:
sentences_score

[(3.0, 0),
 (2.0, 1),
 (3.0, 2),
 (1.0, 3),
 (1.0, 4),
 (1.0, 5),
 (3.0, 6),
 (1.0, 7)]

In [30]:
from IPython.core.display import HTML

In [31]:
text = ''

display(HTML(f'<h2>Summary</h2>'))
for sentence in original_sentences:
  if sentence in best_sentences:
    text += ' ' + sentence.replace(sentence, f"<mark>{sentence}</mark>")
  else:
    text += ' ' + sentence
display(HTML(f""" {text} """))

## **3- Cosine similarity**
- https://en.wikipedia.org/wiki/Cosine_similarity

In [32]:
import re
import nltk
import string
import numpy as np
import networkx as nx
from nltk.cluster.util import cosine_distance

In [33]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [35]:
def preprocess(text):
  formatted_text = text.lower()
  tokens = []
  for token in nltk.word_tokenize(formatted_text):
    tokens.append(token)
  tokens = [word for word in tokens if word not in stopwords and word not in string.punctuation]
  formatted_text = ' '.join(element for element in tokens)

  return formatted_text

In [36]:
original_text = """The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.
                   Another factor for the above-average prediction is the absence of ElNiño, according to CSU researchers.  
                   The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.  
                   This would be the seventh consecutive above-average Atlantic hurricane season.  
                   The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration.  
                   The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc.  
                   Hurricanes have had less impact on energy markets over the years because a greater share of US output comes from onshore.  
                   Storms can disrupt operations at power plants, limiting demand for crude and natural gas, and potentially leading to higher prices for gasoline and other refined products.  
                   The Atlantic hurricane season begins 1 June and ends on 30 November, but storms can form outside of that time. 
                   The CSU Tropical Meterology Project is based on about 40 years of data."""
original_text = re.sub(r'\s+', ' ', original_text)
original_text

"The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today. Another factor for the above-average prediction is the absence of ElNiño, according to CSU researchers. The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook. This would be the seventh consecutive above-average Atlantic hurricane season. The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration. The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc. Hurricanes have had less impact on energy market

In [37]:
formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
formatted_sentences

['2022 atlantic hurricane season could active usual sea temperatures across tropical atlantic higher normal colorado state university csu said annual outlook today',
 'another factor above-average prediction absence elniño according csu researchers',
 'six-month long hurricane season 19 named storms nine could become hurricanes four could turn major hurricanes category 3 higher according outlook',
 'would seventh consecutive above-average atlantic hurricane season',
 'average tropical cyclones atlantic 1991 2020 seven hurricanes three major 14 tropical storms according us national oceanic atmospheric administration',
 "2022 hurricane activity 130pc average season 1991 2020 compared 2021 's hurricane season 120pc",
 'hurricanes less impact energy markets years greater share us output comes onshore',
 'storms disrupt operations power plants limiting demand crude natural gas potentially leading higher prices gasoline refined products',
 'atlantic hurricane season begins 1 june ends 30 nov

In [38]:
def calculate_sentence_similarity(sentence1, sentence2):
  words1 = [word for word in nltk.word_tokenize(sentence1)]
  words2 = [word for word in nltk.word_tokenize(sentence2)]
  #print(words1)
  #print(words2)

  all_words = list(set(words1 + words2))
  #print(all_words)

  vector1 = [0] * len(all_words)
  vector2 = [0] * len(all_words)
  #print(vector1)
  #print(vector2)

  for word in words1: # Bag of words
    #print(word)
    vector1[all_words.index(word)] += 1
  for word in words2:
    vector2[all_words.index(word)] += 1
  
  #print(vector1)
  #print(vector2)

  return 1 - cosine_distance(vector1, vector2)

In [39]:
calculate_sentence_similarity(formatted_sentences[0], formatted_sentences[1])

0.06804138174397711

## **Function to create the similarity matrix**
- The higher the value, the greater the similarity between the sentences
- The more words in common, the greater the similarity

In [40]:
def calculate_similarity_matrix(sentences):
  similarity_matrix = np.zeros((len(sentences), len(sentences)))
  #print(similarity_matrix)
  for i in range(len(sentences)):
    for j in range(len(sentences)):
      if i == j:
        continue
      similarity_matrix[i][j] = calculate_sentence_similarity(sentences[i], sentences[j])
  return similarity_matrix

In [41]:
calculate_similarity_matrix(formatted_sentences)

array([[0.        , 0.06804138, 0.24494897, 0.3086067 , 0.17817416,
        0.24056261, 0.        , 0.04950738, 0.22645541, 0.14433757],
       [0.06804138, 0.        , 0.06666667, 0.12598816, 0.0727393 ,
        0.        , 0.        , 0.        , 0.        , 0.11785113],
       [0.24494897, 0.06666667, 0.        , 0.15118579, 0.21821789,
        0.18856181, 0.11547005, 0.09701425, 0.16641006, 0.        ],
       [0.3086067 , 0.12598816, 0.15118579, 0.        , 0.08247861,
        0.35634832, 0.        , 0.        , 0.31448545, 0.        ],
       [0.17817416, 0.0727393 , 0.21821789, 0.08247861, 0.        ,
        0.15430335, 0.12598816, 0.05292561, 0.12104551, 0.15430335],
       [0.24056261, 0.        , 0.18856181, 0.35634832, 0.15430335,
        0.        , 0.        , 0.        , 0.26148818, 0.        ],
       [0.        , 0.        , 0.11547005, 0.        , 0.12598816,
        0.        , 0.        , 0.        , 0.        , 0.10206207],
       [0.04950738, 0.        , 0.0970142

## **Function to summarize the texts**

- Pagerank algorithm: https://en.wikipedia.org/wiki/PageRank

In [42]:
for i, score in enumerate(original_sentences):
  print(i, score)

0 The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.
1 Another factor for the above-average prediction is the absence of ElNiño, according to CSU researchers.
2 The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.
3 This would be the seventh consecutive above-average Atlantic hurricane season.
4 The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration.
5 The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc.
6 Hurricanes have had less impact on 

In [43]:
def summarize(text, number_of_sentences, percentage = 0):
  original_sentences = [sentence for sentence in nltk.sent_tokenize(text)]
  formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
  similarity_matrix = calculate_similarity_matrix(formatted_sentences)
  #print(similarity_matrix)

  similarity_graph = nx.from_numpy_array(similarity_matrix)
  #print(similarity_graph.nodes)
  #print(similarity_graph.edges)

  scores = nx.pagerank(similarity_graph)
  #print(scores)
  ordered_scores = sorted(((scores[i], score) for i, score in enumerate(original_sentences)), reverse=True)
  #print(ordered_scores)

  if percentage > 0:
    number_of_sentences = int(len(formatted_sentences) * percentage)

  best_sentences = []
  for sentence in range(number_of_sentences):
    best_sentences.append(ordered_scores[sentence][1])
  
  return original_sentences, best_sentences, ordered_scores

In [44]:
original_sentences, best_sentences, scores = summarize(original_text, 3)

In [45]:
original_sentences

['The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.',
 'Another factor for the above-average prediction is the absence of ElNiño, according to CSU researchers.',
 'The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.',
 'This would be the seventh consecutive above-average Atlantic hurricane season.',
 'The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration.',
 "The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc.",
 'Hurricanes have had les

In [46]:
best_sentences

['The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.',
 'This would be the seventh consecutive above-average Atlantic hurricane season.',
 'The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.']

In [47]:
scores

[(0.15037524816350484,
  'The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.'),
 (0.13524902808221945,
  'This would be the seventh consecutive above-average Atlantic hurricane season.'),
 (0.1337734554906151,
  'The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.'),
 (0.1279682880153172,
  'The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration.'),
 (0.12116182363150062,
  "The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc."),
 (0.119102

## **Function to calculate similarity between sentences**

- Link: https://en.wikipedia.org/wiki/Cosine_similarity
- Step by step calculations: https://janav.wordpress.com/2013/10/27/tf-idf-and-cosine-similarity/

In [48]:
original_sentences = [sentence for sentence in nltk.sent_tokenize(original_text)]
original_sentences

['The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.',
 'Another factor for the above-average prediction is the absence of ElNiño, according to CSU researchers.',
 'The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.',
 'This would be the seventh consecutive above-average Atlantic hurricane season.',
 'The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration.',
 "The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc.",
 'Hurricanes have had les

In [50]:
from IPython.core.display import HTML
text = ''

display(HTML(f'<h2>Summary</h2>'))
for sentence in original_sentences:
  if sentence in best_sentences:
    text += ' ' + sentence.replace(sentence, f"<mark>{sentence}</mark>")
  else:
    text += ' ' + sentence
display(HTML(f""" {text} """))

# **Libraries based Text Summarization**
## **1- Sumy**
- https://pypi.org/project/sumy/

In [52]:
!pip install sumy

     |████████████████████████████████| 94 kB 2.1 MB/s 
     |████████████████████████████████| 10.1 MB 23.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21712 sha256=4150199fd061c058f644858af05ef9249b2540b0def374faa0558d6954d3dbcd
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=b02886adaac542647a55062184a7ec56492dfc67322bb9e1321c98156bcaccad
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built breadability pycountry


In [53]:
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

In [54]:
doc.iloc[0]

"The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.  Another factor for the above-average prediction is the absence of El Niño, according to CSU researchers. The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook. This would be the seventh consecutive above-average Atlantic hurricane season. The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration.  The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc. Hurricanes have had less impact on energy mar

In [55]:
summarizer = TextRankSummarizer()
parser = PlaintextParser.from_string(
#    doc.iloc[0].split("\n", 1)[1],
    doc.iloc[0],
    Tokenizer("english"))
summary = summarizer(parser.document, sentences_count =5)

In [56]:
summary

(<Sentence: The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.>,
 <Sentence: The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.>,
 <Sentence: The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration.>,
 <Sentence: The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc.>,
 <Sentence: The Atlantic hurricane season begins 1 June and ends on 30 November, but storms can form outside of that time.>)

In [57]:
for s in summary:
    print(wrap(str(s)))

The 2022 Atlantic hurricane season could be more active than usual as
sea temperatures across the tropical Atlantic are higher than normal,
Colorado State University (CSU) said in its annual outlook today.
The six-month long hurricane season should have 19 named storms, nine
of which could become hurricanes and four that could turn into major
hurricanes of Category 3 or higher, according to the outlook.
The average for tropical cyclones in the Atlantic between 1991 and
2020 is seven hurricanes, three of them major, and 14 tropical storms,
according to the US National Oceanic and Atmospheric Administration.
The 2022 hurricane activity should be about 130pc above the average
season from 1991 to 2020, compared with 2021's hurricane season that
was up by 120pc.
The Atlantic hurricane season begins 1 June and ends on 30 November,
but storms can form outside of that time.


In [60]:
summarizer = LsaSummarizer()
summary1 = summarizer(parser.document, sentences_count =5)
for s in summary1:
    print(wrap(str(s)))

The 2022 Atlantic hurricane season could be more active than usual as
sea temperatures across the tropical Atlantic are higher than normal,
Colorado State University (CSU) said in its annual outlook today.
The six-month long hurricane season should have 19 named storms, nine
of which could become hurricanes and four that could turn into major
hurricanes of Category 3 or higher, according to the outlook.
Hurricanes have had less impact on energy markets over the years
because a greater share of US output comes from onshore.
The Atlantic hurricane season begins 1 June and ends on 30 November,
but storms can form outside of that time.
The CSU Tropical Meterology Project is based on about 40 years of
data.


In [61]:
from IPython.core.display import HTML
text = ''

display(HTML(f'<h2>Summary1</h2>'))
for sentence in original_sentences:
  if sentence in best_sentences:
    text += ' ' + sentence.replace(sentence, f"<mark>{sentence}</mark>")
  else:
    text += ' ' + sentence
display(HTML(f""" {text} """))

## **2- pysummarization**

- https://pypi.org/project/pysummarization/

In [62]:
!pip install pysummarization

     |████████████████████████████████| 51 kB 305 kB/s 
  Created wheel for pysummarization: filename=pysummarization-1.1.8-py3-none-any.whl size=59453 sha256=19a2948de2bcc8d932772dd73f5e94b3319c1fdfa027e2674d551da54b3c4c92
  Stored in directory: /root/.cache/pip/wheels/da/14/3e/02d15001af23ca877c5149b66280a605e5cdbbe76972598afa
Successfully built pysummarization


In [63]:
from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.tokenizabledoc.simple_tokenizer import SimpleTokenizer
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor
import re

In [64]:
auto_abstractor = AutoAbstractor()
auto_abstractor.tokenizable_doc = SimpleTokenizer()
auto_abstractor.delimiter_list = [".", "\n"]
abstractable_doc = TopNRankAbstractor()

In [65]:
summary = auto_abstractor.summarize(doc.iloc[0], abstractable_doc)

In [66]:
summary

{'scoring_data': [(0, 27.129032258064516),
  (1, 13.235294117647058),
  (2, 27.272727272727273),
  (3, 10.0),
  (4, 24.5),
  (5, 21.333333333333332),
  (6, 1.8),
  (7, 2.0),
  (8, 3.6),
  (9, 2.5714285714285716)],
 'summarize_result': ['The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.\n',
  '  Another factor for the above-average prediction is the absence of El Niño, according to CSU researchers.\n',
  ' The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.\n',
  ' This would be the seventh consecutive above-average Atlantic hurricane season.\n',
  ' The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, a

In [67]:
best_sentences = []
for sentence in summary['summarize_result']:
  #print(sentence)
  best_sentences.append(re.sub(r'\s+', ' ', sentence).strip())

In [68]:
best_sentences

['The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.',
 'Another factor for the above-average prediction is the absence of El Niño, according to CSU researchers.',
 'The six-month long hurricane season should have 19 named storms, nine of which could become hurricanes and four that could turn into major hurricanes of Category 3 or higher, according to the outlook.',
 'This would be the seventh consecutive above-average Atlantic hurricane season.',
 'The average for tropical cyclones in the Atlantic between 1991 and 2020 is seven hurricanes, three of them major, and 14 tropical storms, according to the US National Oceanic and Atmospheric Administration.',
 "The 2022 hurricane activity should be about 130pc above the average season from 1991 to 2020, compared with 2021's hurricane season that was up by 120pc.",
 'Hurricanes have had le

In [69]:
from IPython.core.display import HTML
text = ''

display(HTML(f'<h2>Summary1</h2>'))
for sentence in original_sentences:
  if sentence in best_sentences:
    text += ' ' + sentence.replace(sentence, f"<mark>{sentence}</mark>")
  else:
    text += ' ' + sentence
display(HTML(f""" {text} """))

## **3- BERT**

- https://pypi.org/project/bert-extractive-summarizer/

In [70]:
!pip install bert-extractive-summarizer

     |████████████████████████████████| 4.0 MB 9.6 MB/s 
     |████████████████████████████████| 880 kB 52.6 MB/s 
     |████████████████████████████████| 596 kB 50.1 MB/s 
     |████████████████████████████████| 6.6 MB 42.9 MB/s 
     |████████████████████████████████| 77 kB 4.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=1bede30b32bfe93a86147d5a377f9fa4c0a3ec174e30a4aeaee1865d8a61ab15
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [71]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-yux6kcc8
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-yux6kcc8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4099889 sha256=efe76f686a2dded70a3e68df0720e4d2f26e85750930bc2ff619ad4d92f8bd95
  Stored in directory: /tmp/pip-ephem-wheel-cache-rstm0_yq/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0


In [72]:
from transformers import BigBirdTokenizer

In [73]:
from summarizer import Summarizer

In [74]:
summarizer = Summarizer()
summary = summarizer(doc.iloc[0])

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [75]:
summary

'The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today. Hurricanes have had less impact on energy markets over the years because a greater share of US output comes from onshore.'

In [76]:
summary_tokenized = [sentence for sentence in nltk.sent_tokenize(summary)]
summary_tokenized

['The 2022 Atlantic hurricane season could be more active than usual as sea temperatures across the tropical Atlantic are higher than normal, Colorado State University (CSU) said in its annual outlook today.',
 'Hurricanes have had less impact on energy markets over the years because a greater share of US output comes from onshore.']